# Setup and Load dataset



In [ ]:
%%capture
!pip install elasticsearch==7.14.0
!apt install default-jdk > /dev/null

In [ ]:
!pip install sentence_transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 96.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 66.7 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=9ac22898a29b68a4a8f6e6d52578b4469ac8be99e8c4258dffd7bda8db1fd493
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers


In [ ]:
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [ ]:
!gdown --id 1pxg131HVz6t1lGPDiT70SIf8LnVJlVy4

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1pxg131HVz6t1lGPDiT70SIf8LnVJlVy4
To: /content/medium_data.zip
100% 520k/520k [00:00<00:00, 150MB/s]


In [ ]:
!unzip -o "medium_data.zip"  -d  "/content"

Archive:  medium_data.zip
  inflating: /content/medium_data.csv  


In [ ]:
import os
os.kill(os.getpid(), 9)

## Data Loading

In [ ]:
from datetime import datetime
import os
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

from scipy import sparse
from scipy.sparse import csc_matrix
from sklearn.decomposition import TruncatedSVD

np.random.seed(0)

#### Name of the file which contain all the item properties

In [ ]:
file="medium_data.csv"

###### Run below cell

In [ ]:
content_df  = pd.read_table( file,delimiter=',')

In [ ]:
content_df.head(1)

,id,url,title,subtitle,image,claps,responses,reading_time,publication,date
0,1,https://towardsdatascience.com/a-beginners-gui...,A Beginner’s Guide to Word Embedding with Gens...,NaN,1.png,850,8,8,Towards Data Science,2019-05-30


## Details about dataset

In [ ]:
itemid="id"

In [ ]:
features=['title','subtitle'	]

In [ ]:
allcols=[itemid]
for i in features:
  allcols.append(i)

# Setup

In [ ]:
content_df['NewTag']=""

for i in features:
  content_df[i] = content_df[i].fillna(' ')
for i in features:
  content_df['NewTag']+=(' '+content_df[i])
content_df['NewTag']=content_df['NewTag'].astype(str)

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

In [ ]:
def clean_text(text):
    
    text = text.lower()  # lowercase text
    # replace the matched string with ' '
    text = re.sub( re.compile("\'s"), ' ', text)
    text = re.sub(re.compile("\\r\\n"), ' ', text)
    text = re.sub(re.compile(r"[^\w\s]"), ' ', text)
    return text

In [ ]:
stopwords=set(stopwords.words('english'))

In [ ]:
def tokenizer(sentence, min_words=4, max_words=200, stopwords=stopwords, lemmatize=True):
    
    if lemmatize:
        stemmer = WordNetLemmatizer()
        tokens = [stemmer.lemmatize(w) for w in word_tokenize(sentence)]
    else:
        tokens = [w for w in word_tokenize(sentence)]
    token = [w for w in tokens if (len(w) > min_words and len(w) < max_words
                                                        and w not in stopwords)]
    return tokens    

In [ ]:
content_df['clean'] = content_df['NewTag'].apply(clean_text)
# content_df['token_lem_sentence'] = content_df['clean'].apply(
#         lambda x: tokenizer(x))

# Elastic search set up

In [ ]:
import elasticsearch
from elasticsearch import Elasticsearch
from elasticsearch import helpers


In [ ]:
# Download & extract Elasticsearch 7.0.0

!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.0.0

In [ ]:
# Creating daemon instance of elasticsearch
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.0.0/bin/elasticsearch'], 
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )

In [ ]:
# This part is important, since it takes a little amount of time for instance to load
import time
time.sleep(20)

In [ ]:
%%bash
# If you get 1 root & 2 daemon process then Elasticsearch instance has started successfully
ps -ef | grep elasticsearch

daemon      2763     583  4 05:32 ?        00:00:31 /content/elasticsearch-7.0.0/jdk/bin/java -Xms1g -Xmx1g -XX:+UseConcMarkSweepGC -XX:CMSInitiatingOccupancyFraction=75 -XX:+UseCMSInitiatingOccupancyOnly -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.io.tmpdir=/tmp/elasticsearch-5854607123794414241 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,pid,tags:filecount=32,filesize=64m -Djava.locale.providers=COMPAT -Dio.netty.allocator.type=unpooled -Des.path.home=/content/elasticsearch-7.0.0 -Des.path.conf=/content/elasticsearch-7.0.0/config -Des.distribution.flavor=default 

In [ ]:
# Check if elasticsearch is running
!curl -sX GET "localhost:9200/"

{
  "name" : "a5c99c6333e6",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "ntlVnYbTQkue8WZnf_hN6g",
  "version" : {
    "number" : "7.0.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "b7e28a7",
    "build_date" : "2019-04-05T22:55:32.697037Z",
    "build_snapshot" : false,
    "lucene_version" : "8.0.0",
    "minimum_wire_compatibility_version" : "6.7.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [ ]:
es = Elasticsearch(hosts = [{"host":"localhost", "port":9200}])
# Check if python is connected to elasticsearch
es.ping()

True

# Model train

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')


In [ ]:
corpus_embeddings = model.encode(content_df.clean.values,show_progress_bar=True, convert_to_numpy=True)

Batches:   0%|          | 0/204 [00:00<?, ?it/s]

In [ ]:
corpus_embeddings.shape

(6508, 768)

# Search in elastic search using embedding

In [ ]:
# Define settings & mappings of Elasticsearch index
Settings = {
    "settings":{
        "number_of_shards":1,
        "number_of_replicas":0
    },
    "mappings":{
        "properties":{
            "id":{
                "type":"text"
            },
            "longtext":{
                "type":"text"
            },
            "vector":{
                "type": "dense_vector",
                  "dims": corpus_embeddings.shape[1]
            }
        }
    }
}

In [ ]:
indexname='new_index1'
index_type='_doc'

In [ ]:

index = es.indices.create(index=indexname, ignore=[400,404], body=Settings)


In [ ]:
json_list = []

for text,idx, emb in zip(content_df.clean.values,content_df[itemid].values,corpus_embeddings):
            dic = {}
            dic['_index'] = indexname
            dic['_type'] = index_type
            source = {}
            source[itemid] =idx
            source['longtext'] = text
            source['vector'] = emb
            dic['_source'] = source
            json_list.append(dic)

In [ ]:
# For importing Data to elasticsearch we use elasticsearch's bulk API from elasticsearch.helpers
try:
    res = helpers.bulk(es, json_list)
    print("successfully imported to elasticsearch.")
except Exception as e:
    print(f"error: {e}")

/usr/local/lib/python3.8/dist-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


successfully imported to elasticsearch.


In [ ]:
search_query="graph neural network"

bm25

In [ ]:
# Complicated query
query = es.search(
    index=indexname,
    body={
        "size":20,
        "query":{
            "bool":{
                "must":[
                        {"match":{"longtext":search_query}}
                ],
                # "should":[
                #         {"match":{"":""}}
                # ]
            }
        }
    }
)

output = pd.json_normalize((query['hits']['hits']))
output

,_index,_type,_id,_score,_source.id,_source.longtext,_source.vector
0,new_index1,_doc,h9pMSYYB87FaAq3Y7Lul,12.359951,373,interpretable convolutional neural network,"[-0.2315094769001007, -0.4402959644794464, -0...."
1,new_index1,_doc,FNpMSYYB87FaAq3Y7Lqg,11.859541,2,hands on graph neural networks with pytorch ...,"[-0.4515984356403351, -0.09751088917255402, -0..."
2,new_index1,_doc,9tpNSYYB87FaAq3YQtAJ,11.455088,5860,how to build a simple neural network from scr...,"[-0.5914738774299622, -0.06946942210197449, -0..."
3,new_index1,_doc,59pNSYYB87FaAq3YHcbq,11.369293,3285,visual agnosia a neural network analogy,"[-0.6143883466720581, -0.1767873466014862, -0...."
4,new_index1,_doc,8tpNSYYB87FaAq3YALzP,11.009933,736,neural networks training with approximate log...,"[-0.5157615542411804, -0.5716222524642944, 0.0..."
5,new_index1,_doc,ftpNSYYB87FaAq3YGcRb,11.005751,2668,an introduction to convolutional neural netwo...,"[-0.5782726407051086, -0.20257511734962463, -0..."
6,new_index1,_doc,_dpNSYYB87FaAq3YE8HG,10.931221,2027,how does a neural network make predictions,"[-0.13814114034175873, -0.22558589279651642, -..."
7,new_index1,_doc,hdpNSYYB87FaAq3YAL3Q,10.525656,883,neural network algorithms learn how to trai...,"[-0.5868591666221619, -0.927395761013031, 0.03..."
8,new_index1,_doc,VtpNSYYB87FaAq3YNcz6,10.374919,4676,practical graph neural networks for molecular...,"[-0.5409420132637024, -0.617627739906311, -0.2..."
9,new_index1,_doc,QtpNSYYB87FaAq3YE8PH,10.371246,2352,making your first convolutional neural networ...,"[-0.3200874924659729, -0.012852706015110016, -..."


cosine similarity

In [ ]:
search_embedding = model.encode(search_query)

In [ ]:
  #Sematic search
sem_search = es.search(index=indexname, body={
          "query": {
            "script_score": {
              "query": {
                "match_all": {}
              },
              "script": {
                "source": "cosineSimilarity(params.queryVector, doc['vector']) + 1.0",
                "params": {
                  "vector": search_embedding
                }
              }
            }
          }
        })

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/urllib3/connectionpool.py", line 384, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "/usr/local/lib/python3.8/dist-packages/urllib3/connectionpool.py", line 380, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib/python3.8/http/client.py", line 1348, in getresponse
    response.begin()
  File "/usr/lib/python3.8/http/client.py", line 316, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.8/http/client.py", line 277, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
socket.timeout: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/elasticsearch/connection/http_urllib3.py"

ConnectionTimeout: ignored